In [113]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pymagsac
from time import time
from copy import deepcopy

correspondences = np.loadtxt('../graph-cut-ransac/build/data/rigid_pose_example/rigid_pose_example_points.txt')
gt_pose = np.loadtxt('../graph-cut-ransac/build/data/rigid_pose_example/rigid_pose_example_gt.txt')
threshold = 0.05

print("Number of correspondences loaded = ", str(len(correspondences)))

Number of correspondences loaded =  4138


In [114]:
def verify_magsac(corrs, threshold, sampler_id = 0, use_magsac_plus_plus = True, min_iters=1000, max_iters=5000):    
    n = len(corrs)
    
    pose, mask = pymagsac.findRigidTransformation(
        np.ascontiguousarray(correspondences), 
        probabilities = [],
        min_iters = min_iters,
        max_iters = max_iters,
        sampler = sampler_id,
        use_magsac_plus_plus = use_magsac_plus_plus,
        sigma_th = threshold)
    print (deepcopy(mask).astype(np.float32).sum(), 'inliers found')
    return pose, mask

def tranform_points(corrs, T):
    n = len(corrs)
    points1 = np.float32([corrs[i][0:3] for i in np.arange(n)]).reshape(-1,3)
    points2 = np.float32([corrs[i][3:6] for i in np.arange(n)]).reshape(-1,3)
    
    transformed_corrs = np.zeros((corrs.shape[0], 6))

    for i in range(n):
        p1 = np.append(correspondences[i][:3], 1)
        p2 = p1.dot(T)
        transformed_corrs[i][:3] = p2[:3]
        transformed_corrs[i][3:] = corrs[i][3:]
    return transformed_corrs
    

def calculate_error(gt_pose, est_pose):
    R2R1 = np.dot(gt_pose[:3, :3].T, est_pose[:3, :3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:3, 3] - est_pose[:3, 3])
    
    return err_R, err_t


In [115]:

ground_truth_T = gt_pose[:4, :]

print("MAGSAC")
t = time()
gc_T, gc_mask = verify_magsac(correspondences, threshold, min_iters=5000, max_iters=5000, use_magsac_plus_plus=False)
if gc_T is None:
    gc_T = np.eye(4)
else:
    gc_T = gc_T.T
    
print("Run-time = ", time() - t, ' sec')

err_R, err_t = calculate_error(ground_truth_T, gc_T)

print ('Inlier number = ', np.sum(gc_mask))
print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' cm')

MAGSAC
1378.0 inliers found
Run-time =  0.3371613025665283  sec
Inlier number =  1378
Rotation error =  2.1409347137387247 °
Translation error =  0.07257280276286973  cm


In [116]:
print("MAGSAC++")
t = time()
gc_T, gc_mask = verify_magsac(correspondences, threshold, min_iters=5000, max_iters=5000, use_magsac_plus_plus=True)
if gc_T is None:
    gc_T = np.eye(4)
else:
    gc_T = gc_T.T
    
print("Run-time = ", time() - t, ' sec')

err_R, err_t = calculate_error(ground_truth_T, gc_T)

print ('Inlier number = ', np.sum(gc_mask))
print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' cm')

MAGSAC++
1396.0 inliers found
Run-time =  0.31644225120544434  sec
Inlier number =  1396
Rotation error =  2.0646773119481234 °
Translation error =  0.0860553586934398  cm
